In [29]:
import sys

import geopandas as gpd
import movingpandas as mpd
import pandas as pd
from movingpandas import Trajectory

sys.path.append('/home/diviloper/MobilityDB/PyMEOS/')
from pymeos_cffi.functions import meos_initialize
from pymeos import TGeogPointSeq, TGeogPointInst, TGeomPointInst, TGeomPointSeq, TPointSeq, TPointInst

meos_initialize()

In [31]:
def create_point(row) -> TPointInst:
    return TGeogPointInst(string=f"{row['geometry']}@{row.name}")


def extract_wkt_from_traj(trajectory: Trajectory) -> str:
    res = "["
    for row in trajectory.df.iloc():
        res += f'{row.geometry.wkt}@{row.name},'
    res = res[:-1] + ']'
    return res


def create_sequence(trajectory: Trajectory) -> TPointSeq:
    wkt = extract_wkt_from_traj(trajectory)
    return TGeogPointSeq(string=wkt)

### Read AIS CSV

In [4]:
pdf = pd.read_csv('./aisinput.csv')
gdf = gpd.GeoDataFrame(pdf.drop(['latitude', 'longitude'], axis=1),
                       geometry=gpd.points_from_xy(pdf.longitude, pdf.latitude), crs=4326)
gdf.head()

,t,mmsi,sog,geometry
0,2021-01-08 00:00:00,265513270,0.0,POINT (12.27239 57.05900)
1,2021-01-08 00:00:01,219027804,0.0,POINT (11.86628 55.94244)
2,2021-01-08 00:00:01,265513270,0.0,POINT (12.27239 57.05900)
3,2021-01-08 00:00:03,219027804,0.0,POINT (11.86628 55.94244)
4,2021-01-08 00:00:04,265513270,0.0,POINT (12.27239 57.05900)


In [48]:
ships = mpd.TrajectoryCollection(gdf, 'mmsi', t='t')

In [28]:
ship = ships.trajectories[0]
ship.df.head()

,mmsi,sog,geometry,MEOS Point,speed,distance
t,,,,,,
2021-01-08 00:00:05,219001559,0.1,POINT (9.97551 57.59224),POINT(9.976 57.592)@2021-01-08 00:00:05+01,0.278604,0.000000
2021-01-08 00:00:11,219001559,0.1,POINT (9.97551 57.59226),POINT(9.976 57.592)@2021-01-08 00:00:11+01,0.278604,1.671627
2021-01-08 00:00:18,219001559,0.1,POINT (9.97550 57.59226),POINT(9.976 57.592)@2021-01-08 00:00:18+01,0.111064,0.777446
2021-01-08 00:00:24,219001559,0.1,POINT (9.97550 57.59226),POINT(9.976 57.592)@2021-01-08 00:00:24+01,0.000000,0.000000
2021-01-08 00:00:31,219001559,0.1,POINT (9.97549 57.59225),POINT(9.975 57.592)@2021-01-08 00:00:31+01,0.144470,1.011291


In [47]:
ship.df['MEOS Point'] = ship.df.apply(create_point, axis=1)
ship.df.head()

,mmsi,sog,geometry,MEOS Point,speed,distance
t,,,,,,
2021-01-08 00:00:05,219001559,0.1,POINT (9.97551 57.59224),POINT(9.976 57.592)@2021-01-08 00:00:05+01,0.278604,0.000000
2021-01-08 00:00:11,219001559,0.1,POINT (9.97551 57.59226),POINT(9.976 57.592)@2021-01-08 00:00:11+01,0.278604,1.671627
2021-01-08 00:00:18,219001559,0.1,POINT (9.97550 57.59226),POINT(9.976 57.592)@2021-01-08 00:00:18+01,0.111064,0.777446
2021-01-08 00:00:24,219001559,0.1,POINT (9.97550 57.59226),POINT(9.976 57.592)@2021-01-08 00:00:24+01,0.000000,0.000000
2021-01-08 00:00:31,219001559,0.1,POINT (9.97549 57.59225),POINT(9.975 57.592)@2021-01-08 00:00:31+01,0.144470,1.011291


In [8]:
ship.add_speed(overwrite=True)

In [52]:
ship.add_distance(overwrite=True)
sum(ship.df[ship.distance_col_name])

13922.628896537048

In [56]:
sequence = TGeogPointSeq(instant_list=list(ship.df['MEOS Point']), normalize=False)
sequence.distances

TFloatSeq([0@2021-01-08 00:00:05+01, 1.67163@2021-01-08 00:00:11+01, 2.44907@2021-01-08 00:00:18+01, 2.44907@2021-01-08 00:00:24+01, 3.46036@2021-01-08 00:00:31+01, 4.60437@2021-01-08 00:00:37+01, 5.2989@2021-01-08 00:00:44+01, 5.90721@2021-01-08 00:00:50+01, 8.02857@2021-01-08 00:00:57+01, 9.32205@2021-01-08 00:01:03+01, 10.58616@2021-01-08 00:01:10+01, 10.95902@2021-01-08 00:01:17+01, 11.99994@2021-01-08 00:01:23+01, 12.98249@2021-01-08 00:01:29+01, 13.30935@2021-01-08 00:01:36+01, 13.89439@2021-01-08 00:01:42+01, 15.94898@2021-01-08 00:01:49+01, 16.20181@2021-01-08 00:01:56+01, 17.36899@2021-01-08 00:02:02+01, 17.48036@2021-01-08 00:02:08+01, 17.73319@2021-01-08 00:02:15+01, 18.42984@2021-01-08 00:02:21+01, 19.3288@2021-01-08 00:02:28+01, 19.44017@2021-01-08 00:02:35+01, 20.27514@2021-01-08 00:02:41+01, 21.67562@2021-01-08 00:02:47+01, 22.34651@2021-01-08 00:02:54+01, 23.16198@2021-01-08 00:03:00+01, 24.06343@2021-01-08 00:03:14+01, 24.50891@2021-01-08 00:03:20+01, 24.62852@2021-01-

In [57]:
sequence = TGeogPointSeq(instant_list=list(ship.df['MEOS Point']), normalize=True)
sequence.distance

5202.381848699494

In [51]:
sequence = create_sequence(ship)
sequence.distance

5202.381848699494

In [25]:
ship.df['distance']

t
2021-01-08 00:00:05    0.000000
2021-01-08 00:00:11    1.671627
2021-01-08 00:00:18    0.777446
2021-01-08 00:00:24    0.000000
2021-01-08 00:00:31    1.011291
                         ...   
2021-01-08 23:59:53    0.163430
2021-01-08 23:59:55    0.000000
2021-01-08 23:59:56    0.211167
2021-01-08 23:59:57    0.126411
2021-01-08 23:59:58    0.339421
Name: distance, Length: 48323, dtype: float64